# Homework of Ch4. Image Classification using Convolutional Neural Network
----
This is the homework snippet of TU-ETP-AD1062 Machine Learning Fundamentals.

For more information, please refer to:
https://sites.google.com/view/tu-ad1062-mlfundamentals/

> You do NOT have to build up from nothing, please try your best for the following parts:
> - **Your task: HW4.x.y.**

## HW4.1. Import Packages
----
- Data pre-processing:
    - `os`: Used for path join
    - `sklearn.preprocessing.LabelEncoder`: Convert string-based labels into numeric labels
    - `PIL.Image`: For image file read and manipulation
    - `pandas`: Used for CSV reading and writing
- Models construction:
    - `models.*`, `layers.*`, and `optimizers.*`: For loading related components layers to constructing convolutional neural network
    - `utils.to_categorical`: For converting numerical labels into categorical labels
- Performance evaluation:
    - `sklearn.metrics.zero_one_loss`: Used for accuracy evaluation
    - `sklearn.model_selection.train_test_split`: Divide your data into training and validation set for once, then feed into classifier by yourself, observing the score and confusion matrix
    - `mlfund.plot.PlotMetric`: plot confusion matrix (provided by this repository)

In [ ]:
import os
import numpy as np
import pandas as pd

from PIL import Image

from keras.models import Sequential, Model
from keras.layers import Embedding, Conv1D, Conv2D, MaxPooling2D, GlobalMaxPooling1D, GlobalAveragePooling2D, Flatten, Dense, Dropout, Activation
from keras.optimizers import Adadelta
from keras.utils import to_categorical

from matplotlib import pyplot as plt
from mlfund.plot import PlotMetric

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import zero_one_loss
from sklearn.model_selection import train_test_split
%matplotlib inline

## HW4.2. Data pre-processing
----
The code snippet is used to read image files, with corresponded labels provided in its parent directory, i.e.,
- `buildings`
- `forest`
- `glacier` 
- `mountain`
- `sea`
- `street`

### HW4.2.1. Read Image Files
----
Prepare image files as `X_train`, which is an `numpy.ndarray` instance, and string-based label list `y_train_str`

In [ ]:
labels = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

target_size = (32, 32)

# Training set
X_train = np.empty((0, target_size[0], target_size[1], 3), dtype='uint8')
y_train_str = []
sha1_train = []

for label in labels:
    dir_path_current = os.path.join('data', 'hw4', 'train', label)
    print('Processing %s ...' % dir_path_current)
    
    filelist_img = os.listdir(dir_path_current)
    
    # Images
    X_label_set = np.array([np.array(Image.open(os.path.join(dir_path_current, filename_img)).resize( target_size )) for filename_img in filelist_img])
    X_train = np.append(X_train, X_label_set, axis=0)
    
    # Labels
    y_train_str = y_train_str + ([label] * len(X_label_set))
    
    # SHA1
    sha1_train = sha1_train + [filename_img.split('.')[0] for filename_img in filelist_img ]
    
# Shuffle by SHA1
idx_sorted = np.argsort(sha1_train)

X_train = X_train[idx_sorted, :]
y_train_str = np.array(y_train_str)[idx_sorted]
sha1_train = np.array(sha1_train)[idx_sorted]

In [ ]:
# Testing test
X_test = np.empty((0, target_size[0], target_size[1], 3), dtype='uint8')
sha1_test = []

dir_path_current = os.path.join('data', 'hw4', 'test')
print('Processing %s ...' % dir_path_current)

filelist_img = os.listdir(dir_path_current)
    
# Images
X_test = np.array([np.array(Image.open(os.path.join(dir_path_current, filename_img)).resize( target_size )) for filename_img in filelist_img])

# SHA1
sha1_test = [filename_img.split('.')[0] for filename_img in filelist_img ]

### HW4.2.2. Convert String Label to Numeric Labels
----
Use `LabelEncoder` to convert your string lables into `0`, `1`, `2`, ..., and `5`.

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train_str)

y_train = label_encoder.transform(y_train_str)

display( [ (idx, label) for idx, label in enumerate(label_encoder.classes_) ] )

### HW4.2.3. Show the First 10 images for each Category
----
It provides an overview for you to check the dataset read in

In [ ]:
y_train = np.array(y_train)

for label in np.unique(y_train):
    plt.figure(figsize=(16, 4))
    
    X_label_set = X_train[y_train == label]
    
    for i in range(0, 10):
        plt.subplot(1, 10, i+1)
        plt.imshow(X_label_set[i,:])


## HW4.3. Construct your Classifier
----
Build your classifier, with parameters fine-tuned.

> **Your task: HW4.3.1.**  
> Build your own Neural Network models by Keras framework, try to maximize the performance


In [ ]:
def create_convNet(num_classes):
    model = Sequential()

    model.add(Conv2D(100, kernel_size=(3,3), activation='relu', input_shape=(target_size[0], target_size[1], 3)))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(50, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(50,activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(6,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=Adadelta(), metrics=['accuracy'])
    
    return model

In [ ]:
X1, X2, y1, y2 = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

y1_categorical = to_categorical(y1)
y2_categorical = to_categorical(y2)

In [ ]:
model = create_convNet(len(label_encoder.classes_))
model.summary()
model.fit(X1, y1_categorical, batch_size=64, epochs=10)

In [ ]:
y2_categorical_predict = model.predict(X2)
y2_predict = np.argmax(y2_categorical_predict, axis=1)

# Error rate
err_01loss = zero_one_loss(y2, y2_predict)
print('Error rate = %2.3f' % err_01loss)

# Confusion matrix of prediction
plot_conf_mat = PlotMetric(figsize=(6,6))
plot_conf_mat.set_labels(label_encoder.classes_.tolist())
plot_conf_mat.confusion_matrix(y2, y2_predict, True)

## HW4.4. Submit to Kaggle InClass
----
> **Your task: HW4.4.**
> 1. Training with full data set `X_train` with the model created by `create_clf`,
> 2. Predict the **unknown** testing data `X_test` by the trained model, then
> 3. Submit your result to Kaggle

**Notice: You got only 2 chances to submit your result every day, which means you should fine-tune your model by cross-validation**

In [ ]:
# Create model and train
model = create_clf()
model.fit(X_train, y_train)

# Predict the testing data
y_test_predict = model.predict(X_test)
y_test_predict_str = label_encoder.inverse_transform(y_test_predict)

## Before you submit
----
Please join the homework 3 competition by **using the Email ended with \@trendmicro.com as your Kaggle InClass team name**.

Type your Email in the variable `my_trendmicro_email_which_is_also_my_team_name` to make sure you've already read this paragraph, then the following code snippet will help you to generate the csv file for submission.

In [ ]:
my_trendmicro_email_which_is_also_my_team_name = ''


import re
assert re.match(r"[^@]+@trendmicro.com", my_trendmicro_email_which_is_also_my_team_name), "Please read the instruction above paragraph carefully"

target_path = 'data/hw04.result.csv'
df_test_label = pd.DataFrame({'id': df_test_feature['id'], 'label': y_test_predict_str})
df_test_label.to_csv(target_path, index=False)

print('Congratulation! Please submit your result \'%s\' to https://www.kaggle.com/t/' % target_path)